### Predicting $SPY with a LSTM Neural Network

##### Using $SPY prices from January 1st, 2020 to January 1st, 2024

In [ ]:
import datetime
import itertools
import math
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import seaborn as sns
import time

%matplotlib inline